In [1]:
from PIL import Image, ImageDraw
from IPython.display import display
from tqdm.notebook import tqdm

In [2]:
insert_size = 512
border_size = 2

scenes = ['car', 'chess', 'dragon', 'sponza']
n_scenes = len(scenes)

depths = ['near', 'mid', 'far']
colors = ['#2ca02c', '#1f77b4', '#ff7f0e']
n_depths = len(depths)

iters = [1, 5, 10, 15, 20, 30, 40, 50]
n_iters = len(iters)

methods = ['retinal_near-iter_50', 'retinal_mid-iter_50', 'retinal_far-iter_50', 'retinal_range-iter_50', 'naive', 'interpolation']
n_methods = len(methods)

resolutions = ['12x140', '24x280', '36x420', '48x560']
n_resolutions = len(resolutions)

coords = [[(1420, 2460), (2268,  840), (3080, 1560)], # car
          [(1892, 1820), (2640, 1260), (1200, 1480)], # chess
          [(1400, 1440), (2140, 2160), (2160, 1540)], # dragon
          [( 300, 2700), (1760, 2392), (1960, 1800)]] # sponza

In [3]:
with tqdm(total=n_scenes * n_depths * (1 + n_resolutions*4)) as pbar:
    for scene_index, scene in enumerate(scenes):
        for depth_index, depth in enumerate(depths):
            # Reference with inserts.
            with Image.open('figures/{}/{}-focus_{}-reference.png'.format(scene, scene, depth)) as img:
                for insert_index in range(n_depths):
                    x, y = coords[scene_index][insert_index]
                    ImageDraw.Draw(img).rectangle((x, y, x + insert_size - 1, y + insert_size - 1), outline=colors[insert_index], width=border_size)
                img.save('figures/retina/{}-focus_{}-reference.png'.format(scene, depth))
                pbar.update()

            for resolution_index, resolution in enumerate(resolutions):
                # Retinal targeted with inserts.
                with Image.open('figures/{}/{}-focus_{}-{}-retinal_{}-iter_50.png'.format(scene, scene, depth, resolution, depth)) as img:
                    for insert_index in range(n_depths):
                        x, y = coords[scene_index][insert_index]
                        ImageDraw.Draw(img).rectangle((x, y, x + insert_size - 1, y + insert_size - 1), outline=colors[insert_index], width=border_size)
                    img.save('figures/retina/{}-focus_{}-{}-retinal_{}.png'.format(scene, depth, resolution, depth))
                    pbar.update()

                # Retinal range with inserts.
                with Image.open('figures/{}/{}-focus_{}-{}-retinal_{}-iter_50.png'.format(scene, scene, depth, resolution, 'range')) as img:
                    for insert_index in range(n_depths):
                        x, y = coords[scene_index][insert_index]
                        ImageDraw.Draw(img).rectangle((x, y, x + insert_size - 1, y + insert_size - 1), outline=colors[insert_index], width=border_size)
                    img.save('figures/retina/{}-focus_{}-{}-retinal_{}.png'.format(scene, depth, resolution, 'range'))
                    pbar.update()

                # Naive with inserts.
                with Image.open('figures/{}/{}-focus_{}-{}-naive.png'.format(scene, scene, depth, resolution)) as img:
                    for insert_index in range(n_depths):
                        x, y = coords[scene_index][insert_index]
                        ImageDraw.Draw(img).rectangle((x, y, x + insert_size - 1, y + insert_size - 1), outline=colors[insert_index], width=border_size)
                    img.save('figures/retina/{}-focus_{}-{}-naive.png'.format(scene, depth, resolution))
                    pbar.update()

                # Interpolation with inserts.
                with Image.open('figures/{}/{}-focus_{}-{}-interpolation.png'.format(scene, scene, depth, resolution)) as img:
                    for insert_index in range(n_depths):
                        x, y = coords[scene_index][insert_index]
                        ImageDraw.Draw(img).rectangle((x, y, x + insert_size - 1, y + insert_size - 1), outline=colors[insert_index], width=border_size)
                    img.save('figures/retina/{}-focus_{}-{}-interpolation.png'.format(scene, depth, resolution))
                    pbar.update()

  0%|          | 0/204 [00:00<?, ?it/s]

In [4]:
references_inserts = []

with tqdm(total=n_scenes * n_depths) as pbar:
    for scene_index, scene in enumerate(scenes):
        references_inserts.append([])
        for depth_index, depth in enumerate(depths):

            x, y = coords[scene_index][depth_index]

            with Image.open('figures/{}/{}-focus_{}-reference.png'.format(scene, scene, depth)) as img:
                insert = img.crop((x, y, x + insert_size, y + insert_size))
                ImageDraw.Draw(insert).rectangle((0, 0, insert_size - 1, insert_size - 1), outline=colors[depth_index], width=border_size)
                references_inserts[scene_index].append(insert)
            
            pbar.update()

  0%|          | 0/12 [00:00<?, ?it/s]

In [5]:
methods_inserts = []

with tqdm(total=n_scenes * n_depths * n_resolutions * n_methods) as pbar:
    for scene_index, scene in enumerate(scenes):
        methods_inserts.append([])
        for depth_index, depth in enumerate(depths):
            methods_inserts[scene_index].append([])
            for resolution_index, resolution in enumerate(resolutions):
                methods_inserts[scene_index][depth_index].append([])
                for method_index, method in enumerate(methods):

                    x, y = coords[scene_index][depth_index]

                    with Image.open('figures/{}/{}-focus_{}-{}-{}.png'.format(scene, scene, depth, resolution, method)) as img:
                        insert = img.crop((x, y, x + insert_size, y + insert_size))
                        ImageDraw.Draw(insert).rectangle((0, 0, insert_size - 1, insert_size - 1), outline=colors[depth_index], width=border_size)
                        methods_inserts[scene_index][depth_index][resolution_index].append(insert)
                    
                    pbar.update()

  0%|          | 0/288 [00:00<?, ?it/s]

In [6]:
iterations_inserts = []

with tqdm(total=n_scenes * n_depths * n_resolutions * n_iters) as pbar:
    for scene_index, scene in enumerate(scenes):
        iterations_inserts.append([])
        for depth_index, depth in enumerate(depths):
            iterations_inserts[scene_index].append([])
            for resolution_index, resolution in enumerate(resolutions):
                iterations_inserts[scene_index][depth_index].append([])
                for iter_index, n_iter in enumerate(iters):
                    iterations_inserts[scene_index][depth_index][resolution_index].append([])

                    x, y = coords[scene_index][depth_index]

                    with Image.open('figures/{}/{}-focus_{}-{}-retinal_{}-iter_{}.png'.format(scene, scene, depth, resolution, depth, n_iter)) as img:
                        insert = img.crop((x, y, x + insert_size, y + insert_size))
                        ImageDraw.Draw(insert).rectangle((0, 0, insert_size - 1, insert_size - 1), outline=colors[depth_index], width=border_size)
                        iterations_inserts[scene_index][depth_index][resolution_index][iter_index].append(insert)

                    with Image.open('figures/{}/{}-focus_{}-{}-retinal_{}-iter_{}.png'.format(scene, scene, depth, resolution, 'range', n_iter)) as img:
                        insert = img.crop((x, y, x + insert_size, y + insert_size))
                        ImageDraw.Draw(insert).rectangle((0, 0, insert_size - 1, insert_size - 1), outline=colors[depth_index], width=border_size)
                        iterations_inserts[scene_index][depth_index][resolution_index][iter_index].append(insert)
                    
                    pbar.update()

  0%|          | 0/384 [00:00<?, ?it/s]

In [7]:
# Each row varies Resolution
# Each column varies Method

img = Image.new(mode='RGB', size=(n_resolutions * (insert_size-border_size) + border_size, (1 + n_methods) * (insert_size-border_size) + border_size), color=(255, 255, 255))

with tqdm(total=n_scenes * n_depths) as pbar:
    for scene_index, scene in enumerate(scenes):
        for depth_index, depth in enumerate(depths):
            img.paste(references_inserts[scene_index][depth_index], ((n_resolutions - 1) * (insert_size-border_size)//2, 0))

            for resolution_index, resolution in enumerate(resolutions):
                for method_index, method in enumerate(methods):
                    img.paste(methods_inserts[scene_index][depth_index][resolution_index][method_index], (resolution_index * (insert_size-border_size), (1 + method_index) * (insert_size-border_size)))
            img.save('figures/inserts/{}-focus_{}-RxM.png'.format(scene, depth))

            pbar.update()

  0%|          | 0/12 [00:00<?, ?it/s]

In [8]:
# Each row varies Resolution
# Each column varies Iteration

img = Image.new(mode='RGB', size=(n_resolutions * (insert_size-border_size) + border_size, n_iters * (insert_size-border_size) + border_size), color=(255, 255, 255))

with tqdm(total=n_scenes * n_depths) as pbar:
    for scene_index, scene in enumerate(scenes):
        for depth_index, depth in enumerate(depths):

            for resolution_index, resolution in enumerate(resolutions):
                for iter_index, n_iter in enumerate(iters):
                    img.paste(iterations_inserts[scene_index][depth_index][resolution_index][iter_index][0], (resolution_index * (insert_size-border_size), iter_index * (insert_size-border_size)))
            img.save('figures/inserts/{}-focus_{}-retinal_{}-RxI.png'.format(scene, depth, depth))

            for resolution_index, resolution in enumerate(resolutions):
                for iter_index, n_iter in enumerate(iters):
                    img.paste(iterations_inserts[scene_index][depth_index][resolution_index][iter_index][1], (resolution_index * (insert_size-border_size), iter_index * (insert_size-border_size)))
            img.save('figures/inserts/{}-focus_{}-retinal_{}-RxI.png'.format(scene, depth, 'range'))

            pbar.update()

  0%|          | 0/12 [00:00<?, ?it/s]